# Week 2 Day 4 — Airline AI Assistant: Full Exercises

This notebook completes all exercises from the Day 4 walkthrough:

- **Exercise 1**: Add a `set_ticket_price` tool so the assistant can update prices
- **Exercise 2**: Pythonic function dispatch — no if/elif chains, the registry routes calls automatically
- **Limitation Fix**: Persist the *full* conversation (including tool call messages) in SQLite so nothing is lost between Gradio turns
- **Business Applications**: Reflection on commercial use-cases

In [17]:
import os
import re
import json
import uuid
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4.1-mini"
openai = OpenAI(
    api_key=openai_api_key,
    base_url="https://openrouter.ai/api/v1",
)

In [18]:
system_message = """
You are a helpful assistant for an airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
You can look up ticket prices, update them when asked, check loyalty points, and check how many tickets a customer has bought.
When a user asks about loyalty points, use lookup_loyalty_points first.
When a user asks how many tickets a customer bought, use lookup_customer_ticket_count first.
"""

## Database Setup

Three tables in one SQLite file:
- `prices` — ticket prices per city
- `loyalty_accounts` — customer loyalty points by customer ID
- `conversation_history` — the **full** message history per session, including tool call and tool response messages that Gradio normally discards

In [ ]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS loyalty_accounts (
            customer_id TEXT PRIMARY KEY,
            points INTEGER NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customer_ticket_counts (
            customer_id TEXT PRIMARY KEY,
            tickets_bought INTEGER NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS conversation_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            session_id TEXT NOT NULL,
            message_json TEXT NOT NULL
        )
    ''')

    # Seed loyalty accounts (idempotent)
    cursor.execute('INSERT OR REPLACE INTO loyalty_accounts (customer_id, points) VALUES (?, ?)', ('CUST1001', 12500))
    cursor.execute('INSERT OR REPLACE INTO loyalty_accounts (customer_id, points) VALUES (?, ?)', ('CUST2042', 4200))
    cursor.execute('INSERT OR REPLACE INTO loyalty_accounts (customer_id, points) VALUES (?, ?)', ('CUST7788', 88000))

    # Seed purchase counts (idempotent)
    cursor.execute('INSERT OR REPLACE INTO customer_ticket_counts (customer_id, tickets_bought) VALUES (?, ?)', ('CUST1001', 6))
    cursor.execute('INSERT OR REPLACE INTO customer_ticket_counts (customer_id, tickets_bought) VALUES (?, ?)', ('CUST2042', 2))
    cursor.execute('INSERT OR REPLACE INTO customer_ticket_counts (customer_id, tickets_bought) VALUES (?, ?)', ('CUST7788', 15))

    conn.commit()

print("Database tables ready.")

## Tool Functions

### Exercise 1 — `set_ticket_price`

The `get_ticket_price` function already existed. Now we add `set_ticket_price` so the assistant can also *update* prices.

### Business application added — `lookup_loyalty_points`

This lets the assistant answer customer loyalty-balance questions from internal account data.

All tool functions return a plain string — that string becomes the `content` of the `role: tool` message sent back to the model.

In [ ]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
    return f"Ticket price to {city} is ${result[0]:.2f}" if result else f"No price data available for {city}"


def set_ticket_price(city, price):
    print(f"DATABASE TOOL CALLED: Setting price for {city} to ${price}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?',
            (city.lower(), price, price)
        )
        conn.commit()
    return f"Price for {city} has been updated to ${price:.2f}"


def lookup_loyalty_points(customer_id):
    print(f"DATABASE TOOL CALLED: Looking up loyalty points for {customer_id}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT points FROM loyalty_accounts WHERE customer_id = ?', (customer_id.upper(),))
        result = cursor.fetchone()
    return (
        f"Customer {customer_id.upper()} has {result[0]} loyalty points."
        if result else
        f"No loyalty account found for customer ID {customer_id.upper()}."
    )


def lookup_customer_ticket_count(customer_id):
    print(f"DATABASE TOOL CALLED: Looking up ticket count for {customer_id}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT tickets_bought FROM customer_ticket_counts WHERE customer_id = ?', (customer_id.upper(),))
        result = cursor.fetchone()
    return (
        f"Customer {customer_id.upper()} has bought {result[0]} tickets."
        if result else
        f"No purchase history found for customer ID {customer_id.upper()}."
    )


# Seed the database with some starting prices
initial_prices = {"london": 799, "paris": 899, "tokyo": 1420, "berlin": 499, "sydney": 2999}
for city, price in initial_prices.items():
    set_ticket_price(city, price)

print("\nSeed data loaded. Verifying:")
print(get_ticket_price("London"))
print(get_ticket_price("Tokyo"))
print(lookup_loyalty_points("CUST1001"))
print(lookup_customer_ticket_count("CUST1001"))

## Tool Descriptions (JSON) and Tools List

Each tool needs a JSON description so the model knows it exists, what it does, and what arguments to pass. The parameter names here **must match** the Python function's argument names exactly — we rely on this for the Pythonic dispatch below.

We now include a business tool: `lookup_loyalty_points(customer_id)`.

In [ ]:
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city the customer wants to travel to",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set or update the price of a return ticket to the destination city. Use this when the user asks to change or set a price.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city whose ticket price should be set or updated",
            },
            "price": {
                "type": "number",
                "description": "The new price of the return ticket in USD",
            },
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

loyalty_points_function = {
    "name": "lookup_loyalty_points",
    "description": "Get a customer's current loyalty points by customer ID.",
    "parameters": {
        "type": "object",
        "properties": {
            "customer_id": {
                "type": "string",
                "description": "Customer ID such as CUST1001",
            },
        },
        "required": ["customer_id"],
        "additionalProperties": False
    }
}

ticket_count_function = {
    "name": "lookup_customer_ticket_count",
    "description": "Get how many tickets a customer has bought by customer ID.",
    "parameters": {
        "type": "object",
        "properties": {
            "customer_id": {
                "type": "string",
                "description": "Customer ID such as CUST1001",
            },
        },
        "required": ["customer_id"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": get_price_function},
    {"type": "function", "function": set_price_function},
    {"type": "function", "function": loyalty_points_function},
    {"type": "function", "function": ticket_count_function},
]

print("Tools registered:", [t["function"]["name"] for t in tools])

## Exercise 2 — Pythonic Function Registry

The naive approach uses an `if / elif` chain that must grow every time a new tool is added:

```python
# Naive — do NOT do this
if tool_call.function.name == "get_ticket_price":
    ...
elif tool_call.function.name == "set_ticket_price":
    ...
```

Instead, we keep a **registry** — a dict that maps tool names to the actual functions. `handle_tool_calls` then dispatches automatically using `**arguments` keyword unpacking. Adding a new tool only requires adding it to the registry and writing its JSON description — no `if` statements needed anywhere.

In [21]:
# Registry — the only place you need to touch when adding a new tool
tool_functions = {
    "get_ticket_price": get_ticket_price,
    "set_ticket_price": set_ticket_price,
    "lookup_loyalty_points": lookup_loyalty_points,
    "lookup_customer_ticket_count": lookup_customer_ticket_count,
}


def handle_tool_calls(message):
    """Dispatch every tool call the model requested.

    Uses the registry for routing — no if/elif chains.
    The model's argument names must match the Python function's parameter names
    so we can unpack them directly with **arguments.
    """
    responses = []
    for tool_call in message.tool_calls:
        func_name = tool_call.function.name
        func = tool_functions.get(func_name)

        if func is None:
            result = f"Error: unknown tool '{func_name}'"
        else:
            arguments = json.loads(tool_call.function.arguments)
            result = func(**arguments)

        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })
    return responses

## Limitation Fix — Full Conversation History in SQLite

**The problem with the original Day 4 approach:**  
Gradio's `history` parameter only contains `user` and `assistant` turns. Tool-call messages (`role: assistant` with `tool_calls`) and tool-response messages (`role: tool`) are silently dropped. The model mostly copes because it can infer from context, but this is fragile and would be unacceptable in production.

**The fix:**  
We ignore Gradio's `history` entirely and load the full message list, including tool messages from our `conversation_history` SQLite table. Every message we send or receive is saved there immediately. Gradio's history is used only for rendering the chat UI.

In [23]:
def save_messages(session_id, messages_to_save):
    """Append a list of message dicts to the DB for this session."""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        for msg in messages_to_save:
            cursor.execute(
                'INSERT INTO conversation_history (session_id, message_json) VALUES (?, ?)',
                (session_id, json.dumps(msg))
            )
        conn.commit()


def load_history(session_id):
    """Return the full ordered message list for this session from the DB."""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'SELECT message_json FROM conversation_history WHERE session_id = ? ORDER BY id',
            (session_id,)
        )
        return [json.loads(row[0]) for row in cursor.fetchall()]


def assistant_message_to_dict(message):
    """Convert an OpenAI assistant message object to a plain serialisable dict.

    The raw SDK object is not JSON-serialisable, so we build a clean dict
    that can be stored in SQLite and fed back to the API as-is.
    """
    d = {"role": "assistant", "content": message.content}
    if message.tool_calls:
        d["tool_calls"] = [
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments,
                },
            }
            for tc in message.tool_calls
        ]
    return d

## Chat Function

Key design decisions:
1. **Ignore Gradio's `history`** — load from DB instead so tool messages are never lost
2. **`while` loop** (not `if`) — supports the model making multiple sequential rounds of tool calls
3. **Every message saved immediately** — system prompt, user turn, assistant tool-call messages, tool responses, final answer all go into SQLite

In [24]:
def chat(message, history, session_id):
    # Load the real conversation (including tool messages) from DB
    messages = load_history(session_id)

    # First turn: seed the system message
    if not messages:
        system_msg = {"role": "system", "content": system_message}
        save_messages(session_id, [system_msg])
        messages = [system_msg]

    # Append and save the user's new message
    user_msg = {"role": "user", "content": message}
    save_messages(session_id, [user_msg])
    messages.append(user_msg)

    # First API call
    # For specific account queries, force the relevant tool so the model cannot skip it.
    loyalty_query = bool(re.search(r"\b(loyalty|points?)\b", message, flags=re.IGNORECASE))
    ticket_count_query = bool(re.search(r"\b(tickets?\s+bought|how many tickets|tickets?)\b", message, flags=re.IGNORECASE))

    if ticket_count_query:
        tool_choice = {"type": "function", "function": {"name": "lookup_customer_ticket_count"}}
    elif loyalty_query:
        tool_choice = {"type": "function", "function": {"name": "lookup_loyalty_points"}}
    else:
        tool_choice = "auto"

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )

    # Keep looping as long as the model wants to call tools
    while response.choices[0].finish_reason == "tool_calls":
        raw_message = response.choices[0].message
        assistant_msg = assistant_message_to_dict(raw_message)
        tool_responses = handle_tool_calls(raw_message)

        # Persist everything before the next API call
        save_messages(session_id, [assistant_msg] + tool_responses)
        messages.append(assistant_msg)
        messages.extend(tool_responses)

        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Save and return the final assistant answer
    final_content = response.choices[0].message.content
    save_messages(session_id, [{"role": "assistant", "content": final_content}])
    return final_content

## Launch the Assistant

A fresh `SESSION_ID` is generated each time this cell runs, so re-running it starts a clean conversation while previous sessions remain in the DB. The `gr.State` component carries the session ID through every Gradio callback without exposing it to the UI.

In [ ]:
def create_chat_fn():
    """Wrap chat() so gr.ChatInterface gets a (message, history) -> str signature,
    while the session_id is captured in the closure and stays constant for the
    lifetime of this Gradio launch.
    """
    session_id = str(uuid.uuid4())
    print(f"New session: {session_id}")

    def _chat(message, history):
        return chat(message, history, session_id)

    return _chat


gr.ChatInterface(
    fn=create_chat_fn(),
    title="FlightAI Assistant",
    description=(
        "Ask about ticket prices, update prices, check loyalty points, or ticket purchase counts. "
        "Try: *'What's cheaper, London or Paris?'*, "
        "*'Set the price to Berlin to $350, then check it.'*, "
        "*'How many loyalty points does CUST1001 have?'*, "
        "or *'How many tickets has CUST1001 bought?'*"
    ),
).launch(inbrowser=True)

## Inspecting the Persisted History

Run this cell after chatting to see every message stored in SQLite — including the tool-call and tool-response messages that Gradio never shows you in the UI.

In [ ]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute(
        'SELECT session_id, message_json FROM conversation_history ORDER BY id DESC LIMIT 20'
    )
    rows = cursor.fetchall()

print(f"Last {len(rows)} messages across all sessions:\n")
for session_id, msg_json in reversed(rows):
    msg = json.loads(msg_json)
    role = msg.get('role', '?').upper()
    if 'tool_calls' in msg:
        tools_requested = [tc['function']['name'] for tc in msg['tool_calls']]
        print(f"[{session_id[:8]}] {role}: <requests tools: {tools_requested}>")
    elif role == 'TOOL':
        print(f"[{session_id[:8]}] {role}: {msg.get('content', '')}")
    else:
        content = (msg.get('content') or '')[:120]
        print(f"[{session_id[:8]}] {role}: {content}")

## Business Applications

Now that tools are wired up properly, we implemented one concrete business application: loyalty account lookup.

### Implemented in this notebook

| Tool | What it enables |
|---|---|
| `lookup_loyalty_points(customer_id)` | Customer support can instantly answer loyalty-balance questions without switching systems |
| `lookup_customer_ticket_count(customer_id)` | Customer support can quickly answer purchase-history count questions |

### Other ideas I can add next

| Tool | What it enables |
|---|---|
| `search_available_flights(origin, destination, date)` | Real-time availability check against a booking API |
| `book_flight(flight_id, passenger_details)` | Complete a purchase — the assistant becomes a booking agent |
| `cancel_booking(booking_ref)` | Self-service cancellations without a human agent |
| `check_in(booking_ref, seat_preference)` | Online check-in flow driven entirely by conversation |
| `get_exchange_rate(currency)` | Show prices in the user's local currency |
| `send_confirmation_email(booking_ref, email)` | Trigger transactional emails from the conversation |

Each of these is just a new Python function + a JSON description + one new entry in `tool_functions`. The `handle_tool_calls` dispatcher and the `while` loop need **zero changes**. That is the real power of the Pythonic registry pattern.

The conversation history fix means the model always has the full context — including which tools it already called and what they returned — making multi-step workflows reliable rather than fragile.